<a href="https://colab.research.google.com/github/arintaauza/Quantum-game/blob/main/Connect_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Connect 4!

Below you will find a working implementation of (classical) Connect 4. Pretty boring, huh? Your job will be to spice things up and make it "quantum".

In the first week, you will need to decide how you are going to add vectors to the game. In particular, some parts of the game should involve qubits. You first task is to decide what part will be enhanced with this addition quantum freedom.

You may also want to consider some [rule variations](https://en.wikipedia.org/wiki/Connect_Four#Rule_variations) on the game.

# New Section

Game rules:

- 2 players choose a colum to place a coin one after the other
- first player to make 4 coins in a row wins

- player can choose to play a "Superposition" coin that will take up 2 spaces, but will only collapse into one of them. (superposition)
- player can choose to collapse instead of playing
- player can choose to entagle with any column, if the coin below colapses into 1 it will be on top of it. (entanglement)

In [ ]:
# https://github.com/zakwalters/Connect4/blob/master/Connect4.py

"""
Unknows:
1. When do we measure?
2. When/how do we go into superposition.
3. 
Requirements:

1. Qubits - 

"""

import random
import time
from copy import copy
import numpy as np
import math
from numpy import linalg as LNG 
from typing import List, Type



  

In [ ]:
import requests
import json
import math
from google.colab import files

class TheQ:

    req_str = 'http://8b851bd5c0ec.ngrok.io/qsim/perform_operation'
    req_str_qasm = 'http://8b851bd5c0ec.ngrok.io/qsim/qasm'


    def __init__(self):
        self._reg_id = None
    
    @property
    def reg_id(self):
        return self._reg_id

    @reg_id.setter
    def reg_id(self, reg_id):
        self._reg_id = reg_id

    def create_circuit(self, qubits, initial_state = 0):
        data = {
            'operation': 'create_circuit',
            'num_qubits': qubits
        }
        result = requests.post(self.req_str, json=data)
        json_obj = json.loads(result.content)
        reg_id = json_obj['result']

        data = {
            'operation': 'set_state',
            'register': reg_id,
            'state': initial_state,
            'complex_value': {'re': 1, 'im': 0}
        }
        result = requests.post(self.req_str, json=data)
        return reg_id

    # Gates.  gate_name and params are specified from the list above
    def gate(self, gate_name, params):
          data = {
              'operation': 'gate',
              'register': self.reg_id,
              'gate': gate_name
          }
          for k in params.keys():
              data[k] = params[k]
          result = requests.post(self.req_str, json=data)
  
    # params contain an array of qubit numbers to measure, 'lq2m': [0,1,3] would measure qubits 0, 1 and 3 and return the 
    # corresponding 3-bit integer value from 0 to 7.
    def measure_qubit(self, params):
        data = {
            'operation': 'measure', 
            'register': self.reg_id,
        }
        for k in params.keys():
            data[k] = params[k]
        result = requests.post(self.req_str, json=data)

        json_obj = json.loads(result.content)
        return json_obj['result']

    # Always call destroy_circuit to terminate simulation (turn off your QC)
    def destroy_circuit(self):
        data = {
              'operation': 'destroy_circuit', 
              'register': self.reg_id
        }
        result = requests.post(self.req_str, json=data)

    # Print allows you to output the current computational state of the machine.
    def print_vector(self):
        data = {
              'operation': 'state_vector', 
              'register': self.reg_id
        }
        result = requests.post(self.req_str, json=data)
        json_obj = json.loads(result.content)
        return json_obj['result']

    def create_gate_from_operation(self, operation):

        pdb.set_trace()

    def run_cirq_circuit(self, circuit):
        """
        Takes a cirq circuit and converts it to run
        on 'TheQ'.
        """
        # get qubits
        qubits = circuit.all_qubits()

        # create circuit
        self.create_circuit(len(qubits))

        for operation in circuit.all_operations():
            self.create_gate_from_operation(operation)

        # get and create gates
    
    def qasm(self, count, print_vector, qasm):
        qasm = self._format_qasm_file(qasm)
        data = {
            'script': qasm,
            'count': count,
            'state_vector': print_vector
        }
        result = requests.post(self.req_str_qasm, json=data) 
        return json.loads(result.content)

    def _format_qasm_file(self, qasm_file):
        return qasm_file.replace("include \"qelib1.inc\";", "")


In [ ]:
# Define and run a circuit (always remember to destroy_circuit when you are done)
# the_q = TheQ(3)
# # reg_id = create_circuit(3,0)
# print("Simulation ID = ", the_q.reg_id)
# the_q.gate('hadamard', {'q': 0})
# the_q.gate('hadamard', {'q': 1})
# the_q.gate('hadamard', {'q': 2})
# c0 = the_q.measure_qubit({'lq2m': [0,1]})
# print("Q Computer returned value = ", c0)
# print("Computer state = ", the_q.print_vector())
# the_q.destroy_circuit()

In [ ]:
class Qubit:
    # Qubit(0) = (1 0)^T
    # Qubit(1) = (0 1)^T
    # Qubit() = (alpha beta)^T s.t |alpha|^2 + |beta|^2 = 1, alpha, beta are randomly chosen

    #-------------------------------------------------------------------------------
    def __init__(self,init=None):
      if init is None: #random
        alpha = (2*np.random.random()-1) + (2*np.random.random()-1) *1j
        beta = (2*np.random.random()-1) + (2*np.random.random()-1) *1j
        norm = math.sqrt(LNG.norm(alpha)**2 + LNG.norm(beta)**2)
        self.qb = np.r_['c', [alpha/norm, beta/norm]]
      elif isinstance(init, int): #|0> or |1>
        self.qb = np.r_['c', [1-init, init]]
      else: #initialized to a vector
        self.qb = init
  
      num_rows, num_cols = self.qb.shape
      self.nb = num_rows #sqrt(self.nb) = #qubits
      self.p = np.arange(self.nb) 
      for i in range(self.nb):
        self.p[i] = LNG.norm(self.qb[i]**2) #probability of outcomes

    def tensor(self,q2): #q1 tensor q2
      return Qubit(np.kron(self.qb,q2.qb))

    def entangle(self,q2):#(CNot. (H tensor I))(q1q2)
      g = Gate()
      q1 = g.H(self)
      return g.CNot(q1,q2)

    def collapse(self):
      st = np.random.choice(np.arange(0, self.nb), p = self.p) 
      state = np.zeros((self.nb, 1))
      state[st,0] = 1
  
      return Qubit(state)

    def validate(self, alpha, beta):
        assert alpha**2 + beta**2 == 1

class Gate:

    def H(self, q):
      H = np.array([[1,1],[1,-1]]) * 1/math.sqrt(2)
      return Qubit(np.matmul(H,q.qb))

    def X(self, q):
      X = np.array([[0,1],[1,0]])
      return Qubit(np.matmul(X,q.qb))

    def Y(self, q):
      pass

    def Z(self, q):
      pass

    def CNot(self, q1,q2):
      cn = np.array([[1,0,0,0],[0,1,0,0], [0,0,0,1], [0,0,1,0]])
      q1q2 = q1.tensor(q2)
      return Qubit(np.matmul(cn, q1q2.qb))

    def SWAP(self, q1, q2):
      pass

    def Toffoli(self, q1,q2,q3):
      pass

    def T(self, q):
      pass
    
    def S(self, q):
      pass

'''    
class Circuit:

    def __init__(gate_set): 
      self.circ = gate_set
'''

'    \nclass Circuit:\n\n    def __init__(gate_set): \n      self.circ = gate_set\n'

In [ ]:
!pip install cirq

     |████████████████████████████████| 389kB 3.9MB/s 
     |████████████████████████████████| 1.5MB 23.5MB/s 
     |████████████████████████████████| 1.3MB 34.4MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
import numpy as np
import scipy as sp
import scipy.linalg
import cirq
from cirq.ops import H, Z, measure
from cirq import Simulator
import pdb


In [ ]:
BOARDSIZE = 7

def valid_input(question: str, options: List[str], rtype: Type = str):
    print(question, f"[Valid options: {options}]")

    while True:
        try:
            value = rtype(input("> ").upper())

        except ValueError:
            print("Invalid type, try again")
            continue

        if value not in options:
            print("Invalid option, try again")
            continue

        return value

In [ ]:
def print_board(board):

    '''Prints the board, along with the number of each column above.
    Returns None.
    '''

    out = [''.join('  {} '.format(n+1) for n in range(BOARDSIZE))]
    out.append('+ - '*BOARDSIZE + '+')
    for y in range(BOARDSIZE, 0, -1):
        next_row = '| '
        for x in range(1, BOARDSIZE+1):
            next_row += board.get((x,y), ' ') + ' | '
        out.append(next_row)
        out.append('+ - '*BOARDSIZE + '+')
    print('\n' + '\n'.join(out) + '\n')           

def get_player_move():
   

    '''Asks the player for their move. Checks that the move is valid,
    and then returns it as an int.
    '''

    '''
    1. Ask player :
        a) Activate superposition
        b) Collapse
    2. If they want to activate superposition: pick 2 columns
       Else pick 1 column
    '''


    move_type = valid_input(question="What kind of move would you like to play? Normal or Superposition?(N/S)?", options=["N","S","C"])
    valid_range = [col for col in range(1, BOARDSIZE+1) if (col, BOARDSIZE) not in board]
    # collaps
    if move_type == "C":
      superboard.collapse()
      return None
    # normal play
    if move_type == "N":
        move = valid_input(question="Choose a column to place your piece into. ", options=valid_range, rtype=int)
        return [move]
    # superposition play
    if move_type == "S":
        move1 = valid_input(
            question="Choose your first superposition column to place your piece into. ", 
            options=valid_range, 
            rtype=int
        )
        move2 = valid_input(
            question="Choose your second superposition column to place your piece into. ", 
            options=[col for col in valid_range if move1 != col or top[col-1] != BOARDSIZE-1], 
            rtype=int
        )
        return [move1, move2]


def get_cpu_move():

    '''Chooses a column for any cpu player to place their piece into.
    Returns an int.
    Currently uses a simple random choice.
    '''

    # Check if there's a winning move this turn.
    
    for move in range(1, BOARDSIZE+1):
        if (move, 7) not in board:
            if try_moves([move]):
                return move

    # Otherwise choose a random column.

    moves = [x+1 for x in range(BOARDSIZE)]
    move = random.choice(moves)
    while (move, BOARDSIZE) in board:
        move = random.choice(moves)
    return move


def drop_piece(board, move, is_superposition = False):

    '''Accepts a dict, board, to which move will be applied, and
    an int, move, and updates the board by placing the player's
    piece in the lowest free spot in that number column.
    Returns the y co-ordinate it is placed at, as an int.
    '''

    '''
    1. If superposition
    '''
    sp = {
        "X": "/",
        "O": "("
    }

    for y in range(1, BOARDSIZE+1):
        if (move, y) not in board: 
            if is_superposition:
                superposition_player = sp[player]
                board[(move, y)] = superposition_player
            else:
                board[(move, y)] = player
            return y            
        
# @TODO: game_won does not work

def game_won(board):
    '''Accepts two ints, x and y, which are the x and y co-ordinates of the
    last move made. Checks to see if there is a winning line of length WINLEN
    on the board by checking every line that includes the last placed piece.
    Returns bool.
    '''  

    player_X = 0
    player_O = 0

    for pos, value in board.items():
        x = pos[0]
        y = pos[1]

        checkrange = range(1-WINLEN, WINLEN)
        horizontal = ''
        vertical = ''
        forwardslash = ''
        backslash = ''
        for n in checkrange:
            horizontal += board.get((x-n,y), ' ')
            vertical += board.get((x,y-n), ' ')
            forwardslash += board.get((x+n,y+n), ' ')
            backslash += board.get((x+n,y-n), ' ')
        lines = [horizontal, vertical, forwardslash, backslash]
        
        win = board[(x,y)]*WINLEN
        
        for line in lines:
            if win in line:
                if board[(x,y)] == "X":
                    player_X = 1
                else:
                    player_O = 1

    if player_X and player_O:
        return 3
    elif player_X:
        return 2
    elif player_O:
        return 1
    else:
        return 0
    

def try_moves(moves):

    '''Accepts a list of ints, moves, and applies them in order to see
    if they result in a win. Returns a bool.
    '''

    temp_board = copy(board)

    for m in moves:
        last_y = drop_piece(temp_board, m)
        if game_won(temp_board):
            return True
    return False

'''
def exist_superposition():
   if (("(" in board.values()) or ("/" in board.values())):
      return True
'''

def exist_superposition():
   if ((sp[player] in board.values())):
      return True

def exist_entanglement():
  if "E" in board.values():
    return True

In [ ]:
# Settings

WINLEN = 4
players = ('X', 'O')
YOU = players[0]
test = False
sp = {
        "X": "/",
        "O": "("
    }
from typing import Tuple, List

# Start

board = {}
turn = random.choice([n for n in range(len(players))])
top = [0,0,0,0,0,0,0]

class SuperBoard():
    """Connect 4 board"""
    def __init__(self):
        self.circuit = cirq.Circuit()
        self.the_q = TheQ()
        self.qubits = {} # key: cirq.NamedQubit(e.g q0,q1), value: (x,y) (position of qubit)
        self.board_qubits = {}
        self.entanglement=[] #[(pos_qubit_X, pos_qubit_O)]
      
    def update_board_value(self, position, value):
        board[position] = value

    def move_qubit_position(self, new_position, old_position):
        board[new_position] = board[old_position]
        for qb,(pos1, pos2)  in self.qubits.items():
            if old_position == pos1:
                self.qubits[qb] = (new_position, pos2)
            elif old_position == pos2:
                self.qubits[qb] = (pos1, new_position)

    def collapse_qubit(self, qubit, result:int):

        pos1, pos2 = self.qubits.get(qubit)
      
        if result:
          pos_xo = pos1
          pos = pos2
        else:
          pos_xo = pos2
          pos = pos1

        old_x, old_y = pos
        if (board[pos_xo] == "/"):
            self.update_board_value(pos_xo, "X")
        elif (board[pos_xo] == "("):
            self.update_board_value(pos_xo, "O")
        else:
            #look at self.entanglement, find out if qubit is in the first or second position
            for qb in self.entanglement:
                if qb[0] == qubit:
                    self.update_board_value(pos_xo, "X")
                    break
                elif qb[1] == qubit:
                    self.update_board_value(pos_xo, "O")
                    break


        for y in range(old_y, top[old_x-1]):
            self.move_qubit_position((old_x, y), (old_x, y+1))

        try:
            board.pop((old_x, top[old_x-1])) 
        except KeyError:
            print(pos)
            print(top)
            print(board)
            raise
        top[old_x-1] = top[old_x-1]-1
       

    def collapse(self):
        # do stuff here relating to collapse

        # add mesaurement gate for each qubit
        # only collapse the current player's qubits + entanglements
        """
        for key in circuit keys:
            Add M gate to each qubit in circuit
        """

        if exist_entanglement: 
          for key in self.entanglement:
            q0 = key[0]
            q1 = key[1]
            self.circuit.append(measure(q0))  # adder the measure in q0
            self.circuit.append(measure(q1)) # adder the measure in q1


        for key in self.qubits:
          if board[self.qubits[key][0]] == sp[player]:
            self.circuit.append(measure(key))
        # run the circuit
        simulator = Simulator()
        result = simulator.run(self.circuit)

        result_dict = {}
        list_key = []

        for key, value in self.qubits.items():
            if board[self.qubits[key][0]] == sp[player] or board[self.qubits[key][0]] == "E":
            
                self.collapse_qubit(key, result.measurements[key.name][0][0])
                list_key.append(key)

        for key in list_key:
            self.qubits.pop(key)
        
            
        # clear the circuit and qubits
        self.circuit = cirq.Circuit()
        self.entanglement = []
        #self.qubits = {}

    def _absolute_difference(self, pos1, pos2) -> bool:
      sum = abs(pos1[0] - pos2[0]) + (pos1[1] - pos2[1])
      if sum == 1: 
        return True
      return False
  
    def find_surrounding_qubits(self, position: List[Tuple[int,int]]):
      print(f"POSITION: {position}")
      print(f"BOARD: {board}")
      print(f"QUBITS: {self.qubits}")
      surrounding_qubit_positions = []
      surrounding_qubits = []

      # iterate over board and check for any matches
      # find all neighbouring un-collapsed qubits

      for pos in position:
          for piece_position, value in board.items():
              if self._absolute_difference(pos, piece_position):
                  if value == sp[players[(turn+1)%2]]:
                      qubit = self.board_qubits[piece_position]
                      surrounding_qubits.append(qubit)

      '''
      for pos in position:
          for piece_position, value in board.items():
              if value == "/" or value == "(":
                  if piece_position != position[0] and piece_position != position[1]:
                      if self._absolute_difference(pos, piece_position):
                          surrounding_qubit_positions.append(piece_position)
    
      # print(f"surrounding_qubit_positions: {surrounding_qubit_positions}")
      for qubit_position in surrounding_qubit_positions:
          qubit = self.board_qubits[qubit_position]
          if qubit not in surrounding_qubits:
              surrounding_qubits.append(qubit)
      '''
      # print(f"surrounding_qubits: {surrounding_qubits}")
      return surrounding_qubits
      
      


      # if match use the board qubit position with the match
      # and look through the qubits list
      # and add the qubit to a list
    

    def add_qubit(self, position: List[Tuple[int,int]]):
      #first check the surrounding if there's a qubit that's not itself/own by player
      #if exist, entangle them

      nb= 0
      for i in range(len(self.qubits)+1):
        if (cirq.NamedQubit("q" + str(i))) not in self.qubits:
          nb=i
          break
      
      qubit_name = "q" + str(nb)
    
      qubit = cirq.NamedQubit(qubit_name)
      self.qubits[qubit] = position
      self.board_qubits[position[0]] = qubit
      self.board_qubits[position[1]] = qubit

      if not self.find_surrounding_qubits(position):#no neighbour qubit               
        self.circuit.append([H(qubit)])
      else: #entangle the qubits

        surrounding_qubit = self.find_surrounding_qubits(position)[0] #only need one neighbour


        self.circuit.append([cirq.CNOT(qubit, surrounding_qubit)])

        if player=="X":
            self.entanglement.append((qubit, surrounding_qubit))
        else:
            self.entanglement.append((surrounding_qubit, qubit))

        for i in range(2):
          self.update_board_value(self.qubits[qubit][i],"E")
          self.update_board_value(self.qubits[surrounding_qubit][i], "E")

      '''
      surrounding_qubits = self.find_surrounding_qubits(position)

      for surrounding_qubit in surrounding_qubits:
        self.circuit.append([cirq.H(surrounding_qubit), cirq.CNOT(qubit, surrounding_qubit)])
      '''

    def draw(self):
      print_board(self.board)

if test:
    testboard()

print_board(board)
superboard = SuperBoard()

# Game Loop

while True:

    player = players[turn % len(players)]

    print("\nIt is {}'s turn.".format(player))

    #check if there's superposition
    

    if player != YOU:

    #@TODO
    #pick random {0,1,2}
    #0 : normal
    #1 : superposition
    #2 : collapse
        moves = []
        
        while True:
            option = random.randint(0,2)
            if (option != 2) or (exist_superposition()):
                break

        if option == 0:
            moves.append(get_cpu_move())
        elif option == 1:
            moves.append(get_cpu_move())
            moves.append(get_cpu_move())
        else:
            superboard.collapse()
      
        time.sleep(0) # Suspense/Annoyance control
        print("CPU:",moves)

    else:
        moves = get_player_move()
    '''
        if not moves:
            print_board(board)
            continue
    '''

    if moves is not None:
        is_superposition = len(moves) > 1
        positions = []

        for new_move in moves:
            last_y = drop_piece(board, new_move, is_superposition)
            positions.append((new_move,last_y))
            top[new_move-1] = last_y
        if is_superposition:
            superboard.add_qubit(positions)
            
    print_board(board)


    # @TODO

    if game_won(board) == 3:    
        print("Both players have won!")
        break
    elif game_won(board) == 2:
        print("{} has won!".format(players[0]))
        break
    elif game_won(board) == 1:
        print("{} has won!".format(players[1]))
        break
    
    if len(board) == BOARDSIZE**2:
        print("No one wins.")
        break

    turn += 1


  1   2   3   4   5   6   7 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +


It is O's turn.

  1   2   3   4   5   6   7 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   |   |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   | ( |   |   |   |   |   | 
+ - + - + - + - + - + - + - +
|   | ( |   |   |   |   |   | 
+ - + - + - + - + - + - + - +


It is 